In [6]:
import pandas as pd
import csv
from surprise import BaselineOnly, Reader, Dataset
import numpy as np
from collections import defaultdict
from sets import Set
import math

#Loading data
#train data
file_path_train = 'Processed_data_train.csv'
reader = Reader(line_format='user item rating timestamp', sep=',')
trainset = Dataset.load_from_file(file_path_train, reader=reader)
trainset = trainset.build_full_trainset()

#test data
file_path_test = 'Processed_data_test.csv'
testfile = open(file_path_test, 'rb')
testset = csv.reader(testfile, delimiter=',')

#Load item data
itemfile = "items.txt"
ifile = open(itemfile,'r')
ilist = [ i for i in ifile.readlines() ]

#Training with algo
algo1 = BaselineOnly()
algo1.train(trainset)

#lists of user item pairs
traind = defaultdict(Set)
testd = defaultdict(Set)
tfile = open(file_path_train,'rb')
tset = csv.reader(tfile, delimiter = ',')
for row in tset:
    traind[str(row[0])].add(str(row[1]))
for row in testset:
    testd[str(row[0])].add(str(row[1]))
print len(traind),len(testd)    



/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:6: DeprecationWarning: the sets module is deprecated
  


Estimating biases using als...
185986 185986


In [25]:
from datetime import datetime
#predictions
#iterating through all users
startTime = datetime.now()
precision = 0
recall = 0
c=0
for k in traind.keys():
    c+=1
    if c%10000==0:
        print 'yo'
    recoset = Set() #Using a set to track the items we got a rating for
    recolist = []   #Using a lis to track the items and the rating and later sort to get top n recos
    for x in testd[k]: # Iterating through all test items of the particular user and add them to our set
        pred = algo1.predict(k, x, r_ui=3.0)
        recoset.add(x)
        #print pred[3]
        recolist.append((x,pred[3]))
    for i in range(1000): # Adding 1000 random items to simulate the unknown environment
        rn = int(math.floor(np.random.random()*len(ilist))) #Getting the random index
        ritem = str(ilist[rn])
        while(ritem in traind[k] or ritem in recoset):
            rn = int(math.floor(np.random.random()*len(ilist))) #Getting the random index
            ritem = str(ilist[rn])
        pred = algo1.predict(k, ritem, r_ui=3.0)
        recoset.add(ritem)
        recolist.append((ritem,pred[3]))
    recolist.sort(key=lambda x: x[1],reverse=True)
    toplist = Set()
    for i in range(10):
        toplist.add(recolist[i][0])    
    intlist = toplist & testd[k]
    #Precision
    precision += len(intlist)*1.0/10
    #Recall
    recall += len(intlist)*1.0/len(testd[k])
    #break
precision /=len(traind)
recall /=len(traind)
fm = 2*precision*recall/(precision + recall)
print datetime.now() - startTime

yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
0:26:02.612456


In [26]:
print precision, recall, fm
'''
Problems with metrics
Precision can't really measure. So, 
'''

0.103966965255 0.727927926305 0.181947162293


"\nProblems with metrics\nPrecision can't really measure. So, \n"